# IC 4.10 Task Part 1 Steps 5 through 10

## This script contains the following contents:

### Step 5 - Create a profiling variable
### Step 6 - Create visualization to show distribution of profiles
### Step 7 - Aggregate the max, mean, and min variables on a customer-profile level for usage and frequency expenditure
### Step 8 - Compare profiles with regions and departments and identify any links
### Step 9 - Produce suitable charts
### Step 10 - Finalize script and export final data set

In [1]:
# Import Libraries

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import matplotlib.colors as mcolors
from matplotlib.patches import Rectangle

In [2]:
# Set Path

path = r'C:\Users\Josh Wattay\anaconda3\Instacart Basket Analysis'

In [3]:
# Import Data

insta_no_low_activity = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'insta_no_low_activity.pkl'))

## Step 5 - Create a profiling variable

In [4]:
# Create subset sample of data

profile_categories = ['user_id', 'order_number', 'orders_day_of_week',
       'orders_time_of_day', 'days_since_prior_order', 'product_id', 'product_name',
       'add_to_cart_order', 'department_id', 'prices', 'price_range_loc', 
       'busiest_days', 'busiest_period_of_day', 'max_order', 'loyalty_flag',
       'average_purchase_cost', 'spending_flag', 'median_days_btwn_orders',
       'order_frequency_flag', 'Gender', 'State', 'Region', 'Age', 'date_joined',
       'n_dependants', 'marital_status', 'income']

In [5]:
profile_sample_subset = insta_no_low_activity[profile_categories]

In [6]:
# Check output and shape

profile_sample_subset.head()

,user_id,order_number,orders_day_of_week,orders_time_of_day,days_since_prior_order,product_id,product_name,add_to_cart_order,department_id,prices,...,median_days_btwn_orders,order_frequency_flag,Gender,State,Region,Age,date_joined,n_dependants,marital_status,income
0,1,1,2,8,NaN,196,Soda,1,7,9.0,...,20.5,Non-frequent customer,Female,Alabama,South,31,2/17/2019,3,married,40423
1,1,2,3,7,15.0,196,Soda,1,7,9.0,...,20.5,Non-frequent customer,Female,Alabama,South,31,2/17/2019,3,married,40423
2,1,3,3,12,21.0,196,Soda,1,7,9.0,...,20.5,Non-frequent customer,Female,Alabama,South,31,2/17/2019,3,married,40423
3,1,4,4,7,29.0,196,Soda,1,7,9.0,...,20.5,Non-frequent customer,Female,Alabama,South,31,2/17/2019,3,married,40423
4,1,5,4,15,28.0,196,Soda,1,7,9.0,...,20.5,Non-frequent customer,Female,Alabama,South,31,2/17/2019,3,married,40423


In [7]:
profile_sample_subset.shape

(30964564, 27)

In [8]:
# Export Sample Subset

profile_sample_subset.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'profile_sample_subset.pkl'))

In [9]:
# Create random sample starting with seed and boolean list

np.random.seed(4)
df_profile_sample = np.random.rand(len(profile_sample_subset)) <= 0.7

In [10]:
# Place majority 70% of sample in dataframe

big = profile_sample_subset[df_profile_sample]

In [ ]:
# Place minority 30% of sample in dataframe

small = profile_sample_subset[~df_profile_sample]

In [ ]:
# Export Randomized Sample and save as pkl file

small.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'random_sample_for_profiling.pkl'))

In [ ]:
# Check output and shape

small.head()

In [ ]:
small.shape

## Step 6 - Create visualization to show distribution of profiles

#### I will start by creating an 'income_tier' variable, then a 'dependants_flag', followed by an 'age_group' variable.
#### All three will have visualizations to demonstrate the proportional distribution of each group in the data.

In [ ]:
small['income'].describe()

In [ ]:
# Fix formatting to remove scientific notation

pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [ ]:
small['income'].describe()

In [ ]:
# Define thresholds for income tiers

low_income_tier_1_threshold = 30000
low_income_tier_2_threshold = 59850
mid_income_tier_1_threshold = 85000
mid_income_tier_2_threshold = 100000
high_income_tier_1_threshold = 150000
high_income_tier_2_threshold = 200000

In [ ]:
# Create Income Tier column/variable using customers' current 'income' values

small['income_tier'] = pd.cut(
    small['income'],
    bins=[-float('inf'), low_income_tier_1_threshold, low_income_tier_2_threshold, mid_income_tier_1_threshold, mid_income_tier_2_threshold, high_income_tier_1_threshold, float('inf')],
    labels=['Low Income Tier 1', 'Low Income Tier 2', 'Mid Income Tier 1', 'Mid Income Tier 2', 'High Income Tier 1', 'High Income Tier 2'],
    include_lowest=True)

In [ ]:
# Check output

small.head()

In [ ]:
# Check shape

small.shape

In [ ]:
# Check value counts

small['income_tier'].value_counts()

In [ ]:
# Create Bar Chart

income_tier_frequency = (small['income_tier'].value_counts(normalize=True) * 100).sort_index()
income_tier_frequency.plot.bar(color=['green', 'lightgreen', 'lime', 'yellowgreen', 'chartreuse', 'darkgreen'])
plt.xlabel('Income Tier')
plt.ylabel('Percent')
plt.title('Income Tier Ratio')

In [ ]:
# Save as figure

income_tier_frequency.figure = (small['income_tier'].value_counts(normalize=True) * 100).sort_index()
income_tier_frequency.plot.bar(color=['green', 'lightgreen', 'lime', 'yellowgreen', 'chartreuse', 'darkgreen'])
plt.xlabel('Income Tier')
plt.ylabel('Percent')
plt.title('Income Tier Ratio')

In [ ]:
# Save the plot as a PNG file

plt.savefig(os.path.join(path, '04 Analysis','Visualizations', 'income_tier_frequency_bar.png'))

In [ ]:
# Create Pie Chart for alternative visualization

plt.figure(figsize=(15, 15))  # Increase figure size
pie_income_tier = plt.pie(income_tier_frequency, labels=income_tier_frequency.index, autopct='%1.1f%%', colors=['green', 'lightgreen', 'lime', 'yellowgreen', 'chartreuse', 'darkgreen'])
plt.title('Income Tier Ratio')
plt.savefig(os.path.join(path, '04 Analysis','Visualizations', 'income_tier_frequency_pie.png')) # Save figure
plt.show()

### Our High Income Tier 1 customers make up the majority of our consumer base by a noteable margin (13% ahead of Mid Income Tier 1). 
### The High Income Tiers (1 & 2) make up 45.9% of our consumer base. 
### The Mid Income Tiers (1 & 2) make up 35.5% of our consumer base.
### The Low Income Tiers (1 & 2) make up 18.6% of our consumer base.
### It is imperative to explore the behaviors and spending patters of these groups, while also exploring why Instacart is attractive to High Income Tier 1 consumers.

Next we will create a dependants flag.

In [ ]:
# Create flag to identify customers with dependants

small['dependant_mean'] = small.groupby(['user_id'])['n_dependants'].transform('mean')

In [ ]:
# Set parameters for the flag

small.loc[small['dependant_mean'] > 0, 'dependant_flag'] = 'With Dependants'

In [ ]:
small.loc[small['dependant_mean'] <= 0, 'dependant_flag'] = 'Without Dependants'

In [ ]:
# Check output

small['dependant_flag'].value_counts(dropna = False)

In [ ]:
# Check sum to verify total records = to sample subset total records

6969148 + 2319320

In [ ]:
small.shape

Now I will create an Age Group variable to differentiate between young-adult, adult, middle-age, senior, and elderly customers.

In [ ]:
# Check Age Descriptive Statistics

small['Age'].describe()

In [ ]:
# Create the 'age_group' column with default value as 'Unknown'

small['age_group'] = 'Unknown'

In [ ]:
# Assign age groups based on conditions

small.loc[(small['Age'] >= 18) & (small['Age'] <= 28), 'age_group'] = 'Young Adult'
small.loc[(small['Age'] >= 29) & (small['Age'] <= 40), 'age_group'] = 'Adult'
small.loc[(small['Age'] >= 41) & (small['Age'] <= 55), 'age_group'] = 'Middle-Aged'
small.loc[(small['Age'] >= 56) & (small['Age'] <= 70), 'age_group'] = 'Senior'
small.loc[(small['Age'] >= 71) & (small['Age'] <= 82), 'age_group'] = 'Elderly'

In [ ]:
# Check value counts output

small['age_group'].value_counts()

In [ ]:
# Normalize Distribution to Create Percentages for Pie Chart

age_group_percent = (small['age_group'].value_counts(normalize=True) * 100).sort_index()

In [ ]:
# Check output

age_group_percent

In [ ]:
# Create Pie Chart 

age_group_pie = plt.pie(age_group_percent, labels=age_group_percent.index, autopct='%1.1f%%', colors=['aqua', '#069AF3', '#0343DF', '#00FFFF', '#029386'])
plt.title('Age Group Ratio')
plt.savefig(os.path.join(path, '04 Analysis', 'Visualizations', 'age_group_pie.png'))

### Middle-Aged and Senior customers (ages 41-70) make up ~ 46.8% of our consumer base.

### Creating the Customer Profile Variable

In [ ]:
# Convert data-type to string in order to allow for concatination in next step

small['income_tier'] = small['income_tier'].astype(str)
small['dependant_flag'] = small['dependant_flag'].astype(str)

In [ ]:
# Create Customer Profile Variable by concatinating income_tier, dependant_flag, and age_group

small['customer_profile'] = small['age_group'] + '_' + small['income_tier'] + '_' + small['dependant_flag']

In [ ]:
# Check output

small.head()

In [ ]:
# Check Shape

small.shape

In [ ]:
# Check value counts for 'customer_profile'

small['customer_profile'].value_counts()

In [ ]:
# Export Sample Data Frame to preserve variables and calculations

small.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'random_sample_with_profiles.pkl'))

#### Now we will explore the newly created customer profile data.

In [ ]:
# Descriptive Statistics for Customer Profile

small['customer_profile'].describe()

In [ ]:
# Revenue by Customer Profile

customer_profile_revenue = small.groupby('customer_profile')['prices'].sum()

In [ ]:
# Sort values

customer_profile_revenue_sorted = customer_profile_revenue.sort_values(ascending=False)

In [ ]:
# Set colors

colors = plt.cm.Blues(np.linspace(0.8, 0.3, len(customer_profile_revenue_sorted)))

In [ ]:
# Set format for numerals

pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [ ]:
# Plot Horizontal Bar Chart for Revenue by Customer Profile

# Set the size of the figure
plt.figure(figsize=(14, 10))  # Adjust width and height as needed

# Plot Horizontal Bar Chart for Revenue by Customer Profile
customer_profile_revenue_bar = customer_profile_revenue_sorted.plot.barh(
    title='Revenue by Customer Profile', xlabel='Revenue in USD', ylabel='Customer Profile', color='#06C2AC')

# Format x-axis tick labels without scientific notation
plt.ticklabel_format(style='plain', axis='x')

plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
# Save Bar Chart

customer_profile_revenue_bar.figure.savefig(os.path.join(path, '04 Analysis','Visualizations', 'customer_profile_revenue_bar.png'))

#### Now that we have revenue data, it is time to determine ordering preferences based on Customer Profile.

In [ ]:
# How frequent do customer profiles order?

customer_profile_order_frequency = small.groupby('customer_profile')['days_since_prior_order'].mean()

In [ ]:
# Sort Values

customer_profile_order_frequency_sorted = customer_profile_order_frequency.sort_values(ascending=True)

In [ ]:
# Set colors

colors = plt.cm.Blues(np.linspace(0.3, 0.8, len(customer_profile_order_frequency_sorted)))

In [ ]:
# Plot Horizontal Bar Chart for Order Frequency by Customer Profile

# Set the size of the figure
plt.figure(figsize=(14, 10))  # Adjust width and height as needed

# Plot Horizontal Bar Chart for Order Frequency by Customer Profile
customer_profile_order_frequency_bar = customer_profile_order_frequency_sorted.plot.barh(
    title='Order Frequency by Customer Profile', xlabel='Days Since Prior Order', ylabel='Customer Profile', color='#06C2AC')

# Format x-axis tick labels without scientific notation
plt.ticklabel_format(style='plain', axis='x')

plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
# Save Horizontal Bar Chart

customer_profile_order_frequency_bar.figure.savefig(os.path.join(path, '04 Analysis','Visualizations', 'customer_profile_order_frequency_bar.png'))

#### Now that we have frequency, what time of day do customer profiles use Instacart?

In [ ]:
# Create new customer_profile_order_hours using .groupby and relevant columns

customer_profile_order_hours = small.groupby(['customer_profile', 'orders_time_of_day']).size().unstack()

In [ ]:
# We need the most common hour for each group, or the mode, so calculate for mode.

customer_profile_order_hours_mode = small.groupby('customer_profile')['orders_time_of_day'].apply(lambda x: x.mode().iloc[0])

In [1]:
# Now we need a dataframe to plot the information

mode_data_plot = pd.DataFrame({'customer_profile': customer_profile_order_hours_mode.index, 'mode_orders_time_of_day': customer_profile_order_hours_mode.values})

# Plot Horizontal Bar Chart

ax = mode_data_plot.plot(kind='barh', x='customer_profile', y='mode_orders_time_of_day', color='#06C2AC', legend=False, title='Mode Orders Time of Day', xlabel='Mode Hour', ylabel='Customer Profile', width=0.8)

# Account for Size
fig = plt.gcf()
fig.set_size_inches(14, 10)  # Set the desired size

# Display the plot
plt.tight_layout()
plt.show()

NameError: name 'pd' is not defined

In [ ]:
# Save as Figure

customer_profile_order_hours_mode_bar = mode_data_plot.plot(kind='barh', x='customer_profile', y='mode_orders_time_of_day', color='#06C2AC', legend=False, title='Mode Orders Time of Day', xlabel='Mode Hour', ylabel='Customer Profile', width=0.8)

# Account for Size
fig = plt.gcf()
fig.set_size_inches(14, 10)  # Set the desired size

# Display the plot
plt.tight_layout()
plt.show()

In [ ]:
# Save Horizontal Bar Chart 

customer_profile_order_hours_mode_bar.figure.savefig(os.path.join(path, '04 Analysis','Visualizations', 'customer_profile_order_hours_mode_bar.png'))

#### Now that we have the mode for hours orders are placed, what days do loyal customers use Instacart?

In [ ]:
# Plot bar chart for day of week order is placed by customer loyalty flag status

loyalty_day_order_bar = small.groupby('loyalty_flag')['orders_day_of_week'].value_counts().plot.bar(title = 'Order Days by Loyalty Status', xlabel = 'Day of the Week', ylabel = 'Orders Placed', color=['#06C2AC'] * 7 + ['blue'] * 7 + ['teal'] * 7)
plt.tight_layout()

In [ ]:
# Save Horizontal Bar Chart 

loyalty_day_order_bar.figure.savefig(os.path.join(path, '04 Analysis','Visualizations', 'loyalty_day_order_bar.png'))

### Departments

In [ ]:
# Define Departments with a dictionary

department_dictionary = {
                            1: 'frozen',
                            2: 'other',
                            3: 'bakery',
                            4: 'produce',
                            5: 'alcohol',
                            6: 'international',
                            7: 'beverages',
                            8: 'pets',
                            9: 'dry goods pasta',
                            10: 'bulk',
                            11: 'personal care',
                            12: 'meat seafood',
                            13: 'pantry',
                            14: 'breakfast',
                            15: 'canned goods',
                            16: 'dairy eggs',
                            17: 'household',
                            18: 'babies',
                            19: 'snacks',
                            20: 'deli',
                                                    }

In [ ]:
# Form the department name column and be sure to account for any missing values

small['department_name'] = small['department_id'].map(department_dictionary).fillna("missing")

In [ ]:
# Check output

small['department_name'].value_counts()

In [ ]:
# Now we will look at which departments loyal customers are shopping 

# Groupby

df_loyal_dept_group = small.groupby(['department_name', 'loyalty_flag']).size().unstack()

In [ ]:
# Sort Values

df_loyal_dept_group = df_loyal_dept_group.sort_values(by = 'Regular customer', ascending = False)

In [ ]:
# Use a 100% Stacked Bar Chart to plot results
colors = ['#06C2AC', 'blue', 'teal']
loyal_dept_group_bar = df_loyal_dept_group.plot(kind='bar', stacked=True, color=colors)

plt.title('Department Preference by Customer Loyalty')
plt.xlabel('Department')
plt.ylabel('Order Count')

# Set y-axis tick format to integer (no scientific notation)
loyal_dept_group_bar.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))

plt.tight_layout()
plt.show()

#### Produce is the strongest peforming department in terms of sheer order count. Perhaps this is due to customers wanting their produce to be delivered fresh and saving the hassle of driving to the store and bringing it straight back home to make one trip just for produce alone. This should be explored further.
#### Additionally, perhaps there should be incentives for bulk orders, such as discounts on next order, to improve bulk sales.

In [ ]:
# Save Stacked Bar Chart

loyal_dept_group_bar.figure.savefig(os.path.join(path, '04 Analysis','Visualizations', 'loyal_dept_group_bar.png'))

## Step 7 - Aggregate the max, mean, and min variables on a customer-profile level for usage and frequency expenditure

In [ ]:
# For aggregate profiles we need to use the user_id to group by the relevant columns

aggregate_customer_profile = small.groupby('user_id').agg({
    'Age': ['max', 'mean', 'min'],
    'n_dependants': ['max', 'mean', 'min']
}).reset_index()

In [ ]:
# Rename Columns for clarification of meaning

aggregate_customer_profile.columns = ['user_id', 'max_age', 'mean_age', 'min_age', 'max_dependants', 'mean_dependants', 'min_dependants']

In [ ]:
# Check Output

print(aggregate_customer_profile)

## Step 8 - Compare profiles with regions and departments and identify any links

### Comparing Region and Department

In [ ]:
# Start by comparing Department results by Region

df_dept_by_region = small.groupby(['department_name', 'Region']).size().unstack()

In [ ]:
# Sort Values

df_dept_by_region = df_dept_by_region.sort_values(by = 'Midwest', ascending = False)

In [ ]:
# Chart data with 100% Stacked Bar

colors = ['aqua', '#069AF3', 'teal', 'blue']
dept_by_region_bar = df_dept_by_region.plot(kind='bar', stacked=True, color=colors)
plt.title('Department Outcomes by Region')
plt.xlabel('Department')
plt.ylabel('Order Count')

# Remove scientific notation from y-axis
dept_by_region_bar.yaxis.set_major_formatter('{:.0f}'.format)

plt.tight_layout()
plt.show()

In [ ]:
# Save Bar Chart

dept_by_region_bar.figure.savefig(os.path.join(path, '04 Analysis','Visualizations', 'dept_by_region_bar.png'))

### Observing Region and Order Day of Week

In [ ]:
# Plot Bar Chart to observe Regional preferences for which Day of the Week to place orders

region_order_day_bar = small.groupby('Region')['orders_day_of_week'].value_counts().plot.bar(title = 'Day of Order by Region', xlabel = 'Day of Week', ylabel = 'Order Count',color=['aqua'] * 7 + ['#069AF3'] * 7+['teal'] * 7 + ['blue'] * 7)

plt.tight_layout()

#### It is important to note that each Region conducts the most orders on Saturday (0) followed closely by Sunday (1). 
#### Conversely, each Regions slowest days for orders are Wednesday (4) and Tuesday (3).

In [ ]:
# Save Bar Chart

region_order_day_bar.figure.savefig(os.path.join(path, '04 Analysis','Visualizations', 'region_order_day_bar.png'))

### Observing the most popular products

In [ ]:
# Aggregate the sum of the most popular products in new dataframe

popular_prods = small.groupby('product_name')['order_number'].sum()

In [ ]:
# Sort Values

popular_prods = popular_prods.sort_values(ascending=False)

In [ ]:
# Set colors

colors = plt.cm.Blues(np.linspace(0.8, 0.3, len(popular_prods)))

In [ ]:
# Create dataframe with only the top 10 products

top_10_products = popular_prods.head(10)

In [ ]:
# Plotting
top_10_products_bar = top_10_products.plot.bar(title='Top 10 Products',
                                              xlabel='Product',
                                              ylabel='Order Count',
                                              color='skyblue')
# Remove scientific notation from y-axis
top_10_products_bar.ticklabel_format(style='plain', axis='y')

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
# Save Bar Chart

top_10_products_bar.figure.savefig(os.path.join(path, '04 Analysis','Visualizations', 'top_10_products_bar.png'))

In [ ]:
# Create dataframe with bottom 10 products

bottom_10_products = popular_prods.tail(10)

In [ ]:
# Plotting
bottom_10_products_bar = bottom_10_products.plot.bar(title='Bottom 10 Products',
                                              xlabel='Product',
                                              ylabel='Order Count',
                                              color='skyblue')
# Remove scientific notation from y-axis
bottom_10_products_bar.ticklabel_format(style='plain', axis='y')

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
# Save Bar Chart

bottom_10_products_bar.figure.savefig(os.path.join(path, '04 Analysis','Visualizations', 'bottom_10_products_bar.png'))

### Observing Average Price of Purchase by Region

In [ ]:
# Create a cross-tabulation of average price by Region

avg_price_by_region = pd.crosstab(index=small['Region'], columns='Average Price', values=small['prices'], aggfunc='mean')

In [ ]:
# Plotting
avg_price_by_region_bar = avg_price_by_region.plot(kind='bar', color='skyblue', legend=None)
plt.title('Average Price by Region')
plt.xlabel('Region')
plt.ylabel('Average Price')

# Remove scientific notation from y-axis
plt.gca().ticklabel_format(style='plain', axis='y')

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
# Check output of crosstab for clarity

avg_price_by_region

In [ ]:
# Save Bar Chart

avg_price_by_region_bar.figure.savefig(os.path.join(path, '04 Analysis','Visualizations', 'avg_price_by_region_bar.png'))

### Observing Average Price by Income Tier

In [ ]:
# Create a cross-tabulation of average price by income tier

avg_price_by_income_tier = pd.crosstab(index=small['income_tier'], columns='Average Price', values=small['prices'], aggfunc='mean')

In [ ]:
# Plotting
avg_price_by_income_bar = avg_price_by_income_tier.plot(kind='bar', color='skyblue', legend=None)
plt.title('Average Price by Income Tier')
plt.xlabel('Income Tier')
plt.ylabel('Average Order Price')

# Remove scientific notation from y-axis
plt.gca().ticklabel_format(style='plain', axis='y')

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
# Check output of crosstab for clarity

avg_price_by_income_tier

In [ ]:
# Save Bar Chart

avg_price_by_income_bar.figure.savefig(os.path.join(path, '04 Analysis','Visualizations', 'avg_price_by_income_bar.png'))

### Observing Orders Day of Week by Income

In [ ]:
# Create a cross-tabulation of average orders day of week by income tier

avg_orders_day_by_income_tier = pd.crosstab(index=small['income_tier'], columns='Average Orders Day', values=small['orders_day_of_week'], aggfunc='mean')

In [ ]:
# Plotting
avg_orders_day_by_income_tier_bar = avg_orders_day_by_income_tier.plot(kind='bar', color='skyblue', legend=None)
plt.title('Average Order Day by Income Tier')
plt.xlabel('Income Tier')
plt.ylabel('Average Order Day of Week')

# Remove scientific notation from y-axis
plt.gca().ticklabel_format(style='plain', axis='y')

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
# Check output of crosstab for clarity

avg_orders_day_by_income_tier

In [ ]:
# Save Bar Chart

avg_orders_day_by_income_tier_bar.figure.savefig(os.path.join(path, '04 Analysis','Visualizations', 'avg_orders_day_by_income_tier_bar.png'))

### Observing Regional Ordering Habits

In [ ]:
# Create Line Chart

sns.set_theme(style="whitegrid")  # Set the seaborn theme if needed
regional_ordering_habits_line = sns.lineplot(data=small, x='orders_time_of_day', y='order_number', hue='Region')

# Define title and labels
plt.title('Regional Ordering Habits')
plt.xlabel('Time of Day')
plt.ylabel('Orders')

# Display the legend
plt.legend(title='Region', loc='upper right')

# Show the plot
plt.show()

In [ ]:
#Save Line Chart

regional_ordering_habits_line.figure.savefig(os.path.join(path, '04 Analysis','Visualizations', 'regional_ordering_habits_line.png'))

### A clear trend has emerged, demonstrating that each region seemingly has an ordering peak at ~ 7-8AM and an ordering trough at ~ 1-3AM.
### There is an additional nadir at 7PM (hour 19) and a slight rebound to another mini-peak in orders at 9PM (hour 21).

Let's create a line chart observing the relationship between order number and day of the week, by region.

In [ ]:
# Create Line Chart

sns.set_theme(style="whitegrid")  # Set the seaborn theme if needed
regional_ordering_day_habits_line = sns.lineplot(data=small, x='orders_day_of_week', y='order_number', hue='Region')

# Define title and labels
plt.title('Regional Ordering Habits by Week')
plt.xlabel('Day of Week')
plt.ylabel('Orders')

# Display the legend
plt.legend(title='Region', loc='upper right')

# Show the plot
plt.show()

### It is important to note that the order number per user increases significantly from Saturday (0) to Sunday (1) then it slowly increases as the week progresses until dramatically decreasing on Friday (6).

In [ ]:
#Save Line Chart

regional_ordering_day_habits_line.figure.savefig(os.path.join(path, '04 Analysis','Visualizations', 'regional_ordering_day_habits_line.png'))

#### Let's observe the relationship between the new price range groupings for items by Region.

In [ ]:
# Create a cross-tabulation of count of 'price range loc' categories by Region
price_range_loc_by_region = pd.crosstab(index=small['Region'], columns=small['price_range_loc'])

In [ ]:
# Plotting
price_range_loc_by_region_bar = price_range_loc_by_region.plot(kind='bar', stacked=True)
plt.title('Count of Price Range Categories by Region')
plt.xlabel('Region')
plt.ylabel('Count')

# Remove scientific notation from y-axis
plt.gca().ticklabel_format(style='plain', axis='y')

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
# Save Bar Chart

price_range_loc_by_region_bar.figure.savefig(os.path.join(path, '04 Analysis','Visualizations', 'price_range_loc_by_region_bar.png'))

In [ ]:
# Check output to clarify results

price_range_loc_by_region

In [ ]:
# Given cross-tabulation result
data = {
    'High-range product': [28186, 20947, 39766, 30737],
    'Low-range product': [679414, 514775, 965924, 742886],
    'Mid-range product': [1471822, 1103292, 2087231, 1603488]
}

index = ['Midwest', 'Northeast', 'South', 'West']

# Create DataFrame
df = pd.DataFrame(data, index=index)

In [ ]:
# Calculate total orders by region
total_orders_by_region = df.sum(axis=1)

In [ ]:
# Calculate ratios
ratios_df_product_loc = df.div(total_orders_by_region, axis=0)*100 #for percentage instead of decimal

In [ ]:
# Check ouput
ratios_df_product_loc

In [ ]:
# Plotting
ratios_df_product_loc_bar = ratios_df_product_loc.plot(kind='bar', stacked=True)
plt.title('Ratio of Price Range Loc Categories by Region')
plt.xlabel('Region')
plt.ylabel('Ratio')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
# Save Bar Chart

ratios_df_product_loc_bar.figure.savefig(os.path.join(path, '04 Analysis','Visualizations', 'ratios_df_product_loc_bar.png'))

#### The ratio of product price range groups is nearly identical for each region.

## Step 9 - Produce suitable charts

### Observations Re: Age v. Prices

In [ ]:
# Create a cross-tabulation of average price by age group
avg_price_by_age_group = pd.crosstab(index=small['age_group'], columns='Average Price', values=small['prices'], aggfunc='mean')

In [ ]:
# Plotting
avg_price_by_age_group_bar = avg_price_by_age_group.plot(kind='bar', color='skyblue', legend=None)
plt.title('Average Price by Age Group')
plt.xlabel('Age Group')
plt.ylabel('Average Price')

# Remove scientific notation from y-axis
plt.gca().ticklabel_format(style='plain', axis='y')

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
# Check output of crosstab to clarify result

avg_price_by_age_group

In [ ]:
# Save Bar Chart

avg_price_by_age_group_bar.figure.savefig(os.path.join(path, '04 Analysis','Visualizations', 'avg_price_by_age_group_bar.png'))

### Observations Re: Marital/Family Status

In [ ]:
# Check Values
marital_status_counts = small['marital_status'].value_counts()

In [ ]:
marital_status_counts

In [ ]:
# Plotting pie chart 
marital_status_pie = plt.pie(marital_status_counts, labels=marital_status_counts.index, autopct='%1.1f%%', colors=['aqua', '#069AF3', 'blue', 'teal'])

# Adding title
plt.title('Family Status Distribution')

# Save the figure as an image
save_path = os.path.join(path, '04 Analysis', 'Visualizations', 'marital_status_pie.png')
plt.savefig(save_path)

In [ ]:
# Create a dataframe to group the data by 'marital_status' and 'age_group'
df_age_marital = small.groupby(['marital_status', 'age_group']).size().unstack()

In [ ]:
# Set the figure size
plt.figure(figsize=(10, 6))

# Plot the data using a bar chart
colors = ['aqua', 'blue', 'teal', 'skyblue', 'indigo']
age_marital_bar = df_age_marital.plot(kind='barh', stacked=True, color=colors)

# Set the x-axis to display without scientific notation
plt.ticklabel_format(axis='x', style='plain')

plt.title('Orders in Different Age Groups and Marital Status')
plt.xlabel('Number of Customers')
plt.ylabel('Marital Status')

# Rotate x-axis labels
age_marital_bar.set_xticklabels(age_marital_bar.get_xticklabels(), rotation=45)

plt.tight_layout()
plt.savefig(os.path.join(path, '04 Analysis', 'Visualizations', 'age_marital_bar.png'), bbox_inches='tight', pad_inches=0.1)
plt.show()

#### Chart for Marital Status v. Average Price of Order

In [ ]:
# Calculate the average of 'prices' column grouped by 'marital_status'
avg_prices_by_marital_status = small.groupby('marital_status')['prices'].mean()

In [ ]:
# Plotting
avg_prices_by_marital_status_bar = avg_prices_by_marital_status.plot(kind='bar', color='skyblue')
plt.title('Average Prices by Marital Status')
plt.xlabel('Marital Status')
plt.ylabel('Average Price of Order')

# Remove scientific notation from y-axis
plt.gca().ticklabel_format(style='plain', axis='y')

# Show the plot
plt.show()

In [ ]:
# Check output of crosstab to clarify results

avg_prices_by_marital_status

In [ ]:
# Save Bar Chart

avg_prices_by_marital_status_bar.figure.savefig(os.path.join(path, '04 Analysis','Visualizations', 'avg_prices_by_marital_status_bar.png'))

### Observing Brand Loyalty

In [ ]:
# Calculate the frequency distribution of 'loyalty_flag'
loyalty_distribution = small['loyalty_flag'].value_counts()

# Plot the pie chart
plt.pie(loyalty_distribution, labels=loyalty_distribution.index, autopct='%1.1f%%', colors=['teal', 'blue', 'indigo'], textprops={'color': "gray"})
plt.title('Brand Loyalty Distribution')

# Save the plot
plt.savefig(os.path.join(path, '04 Analysis', 'Visualizations', 'brand_loyalty_pie.png'))

# Display the plot
plt.show()

### Observing Department Popularity as a Ratio

In [ ]:
# Calculate the counts of each department
department_counts = small['department_name'].value_counts()

# Identify the top 5 departments
top_departments = department_counts.head(5).index

# Group other departments into "Other"
small['department_grouped'] = small['department_name'].where(small['department_name'].isin(top_departments))

# Calculate the percentage of each department group
department_percentage = small['department_grouped'].value_counts(normalize=True) * 100

# Define shades of blue
shades_of_blue = plt.cm.Blues(np.linspace(0.9, 0.3, len(department_percentage)))

# Create a pie chart with shades of blue
fig, ax = plt.subplots(figsize=(8, 8))
pie = ax.pie(department_percentage, labels=department_percentage.index, autopct='%1.1f%%', startangle=90, colors=shades_of_blue)

# Add a legend with shades of red
ax.legend(pie[0], department_percentage.index, bbox_to_anchor=(1, 0.5), loc="center left", fontsize=10, bbox_transform=plt.gcf().transFigure)

# Set the title
plt.title('Top Performing Departments', )

# Save the figure as an image
save_path = os.path.join(path, '04 Analysis', 'Visualizations', 'top_departments_pie_chart.png')
plt.savefig(save_path)

# Display the pie chart
plt.show()

### Histogram of Order Frequency

In [ ]:
# Order Frequency Histogram
hist_order_frequency = small['days_since_prior_order'].plot.hist(bins=70, title='Order Frequency', xlabel='Days since Prior Order', color='teal')

# Remove scientific notation from y-axis
plt.ticklabel_format(style='plain', axis='y')

# Show the plot
plt.show()

### Most customers have 7 days and 30 days since their prior order, suggesting weekly or monthly shopping on the site. Given this insight, we should be seeing a larger amount of "bulk" department orders.

In [ ]:
# Save Histogram

hist_order_frequency.figure.savefig(os.path.join(path, '04 Analysis','Visualizations', 'hist_order_frequency.png'))

### Bar Chart Displaying Customer Loyalty Distribution

In [ ]:
# Bar Chart Displaying Loyalty Distribution of Customers
order_loyalty_bar = small['loyalty_flag'].value_counts().plot.bar(title='Loyal Customer Distribution', xlabel='Loyalty Rank', ylabel='Number of Customers', color=['teal', 'blue', 'indigo'])

# Remove scientific notation from y-axis
plt.ticklabel_format(style='plain', axis='y')

# Rotate x-axis labels
plt.xticks(rotation=0)

# Show the plot
plt.show()

In [ ]:
# Save Bar Chart

order_loyalty_bar.figure.savefig(os.path.join(path, '04 Analysis','Visualizations', 'order_loyalty_bar.png'))

### Revising Orders Day of Week Bar Chart

In [ ]:
# Calculate the count of orders for each day of the week
orders_by_day_of_week = small['orders_day_of_week'].value_counts()

# Define a spectrum of 7 shades of blue color palette
blue_palette = sns.color_palette('Blues', 7)

# Plot the count of orders as a bar graph
orders_day_of_week_bar = orders_by_day_of_week.plot(kind='bar', color=blue_palette)

# Set the title and labels
plt.title('Count of Orders by Day of Week')
plt.xlabel('Day of Week')
plt.ylabel('Number of Orders')

# Remove scientific notation from y-axis
plt.ticklabel_format(style='plain', axis='y')

# Show the plot
plt.show()

In [ ]:
# Save Bar Chart

orders_day_of_week_bar.figure.savefig(os.path.join(path, '04 Analysis','Visualizations', 'orders_day_of_week_bar.png'))

## Step 10 - Export Final Data Set

In [ ]:
small.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'final_sample_data_set.pkl'))